# Data Retrieval

### 1. All Data for a team will be retrieved via an SQL query, the documentation is provided in the main folder of the repository

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')
import util
import pandas as pd
import tabulate

# Display settings for better notebook visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

db = util.DatabaseConnection()

# Get all team IDs
team_query = "SELECT team_id, team_name FROM teams;"
teams_df = db.execute_query(team_query)

# Initialize a dictionary to store results for each team
all_transition_data = {}

# SQL query template
query_template = """
WITH action_changes AS (
    SELECT
        a.*,
        LAG(a.team_id) OVER (PARTITION BY a.game_id ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
        LEAD(a.team_id) OVER (PARTITION BY a.game_id ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
    FROM
        public.spadl_actions a
),
-- Determine which side each team starts on in the match
team_sides AS (
    SELECT DISTINCT
        game_id,
        FIRST_VALUE(team_id) OVER (PARTITION BY game_id ORDER BY period_id, seconds, id) AS first_team,
        FIRST_VALUE(next_team_id) OVER (PARTITION BY game_id ORDER BY period_id, seconds, id) AS second_team,
        CASE 
            WHEN FIRST_VALUE(end_x) OVER (PARTITION BY game_id ORDER BY period_id, seconds, id) < 52.5
            THEN 'LEFT_TO_RIGHT'
            ELSE 'RIGHT_TO_LEFT'
        END AS game_direction
    FROM action_changes
    WHERE prev_team_id IS NULL  -- First action of the game
),
possession_loss_moments AS (
    SELECT
        ac.id,
        ac.game_id,
        ac.period_id,
        ac.seconds AS loss_time,
        ac.team_id AS team_losing_ball,
        ac.next_team_id AS team_gaining_ball,
        ac.start_x AS loss_x,
        ac.start_y AS loss_y,
        ts.game_direction
    FROM
        action_changes ac
    JOIN
        team_sides ts ON ac.game_id = ts.game_id
    WHERE
        ac.team_id = '{team_id}'
        AND ac.next_team_id IS NOT NULL
        AND ac.next_team_id != ac.team_id
),
actions_after_loss AS (
    SELECT
        plm.id AS loss_event_id,
        plm.game_id,
        plm.period_id,
        plm.loss_time,
        plm.team_losing_ball,
        plm.team_gaining_ball,
        plm.loss_x,
        plm.loss_y,
        plm.game_direction,
        a.id AS action_id,
        a.seconds AS action_time,
        (a.seconds - plm.loss_time) AS seconds_after_loss,
        a.team_id,
        t.team_name,
        a.player_id,
        p.player_name,
        a.action_type,
        a.result,
        a.start_x,
        a.start_y,
        a.end_x,
        a.end_y,
        -- Add indicator for successful defensive actions (combined tackles, interceptions, and clearances)
        CASE 
            WHEN (a.action_type = '9' OR a.action_type = '10' OR a.action_type = '18') 
                 AND a.result = '1' 
            THEN TRUE 
            ELSE FALSE 
        END AS successful_defensive_action,
        -- Determine movement direction considering period and game direction
        CASE 
            -- Period 1 & 3: Teams play in original direction
            WHEN a.period_id IN (1, 3) THEN
                CASE
                    -- Team starts LEFT_TO_RIGHT
                    WHEN plm.game_direction = 'LEFT_TO_RIGHT' AND a.team_id = plm.team_losing_ball THEN
                        CASE
                            WHEN a.start_x < a.end_x THEN 'FORWARD'
                            WHEN a.start_x > a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- Team starts RIGHT_TO_LEFT
                    WHEN plm.game_direction = 'RIGHT_TO_LEFT' AND a.team_id = plm.team_losing_ball THEN
                        CASE
                            WHEN a.start_x > a.end_x THEN 'FORWARD'
                            WHEN a.start_x < a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- For the team that gained possession (opposite direction)
                    WHEN plm.game_direction = 'LEFT_TO_RIGHT' AND a.team_id = plm.team_gaining_ball THEN
                        CASE
                            WHEN a.start_x > a.end_x THEN 'FORWARD'
                            WHEN a.start_x < a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- Team starts RIGHT_TO_LEFT
                    WHEN plm.game_direction = 'RIGHT_TO_LEFT' AND a.team_id = plm.team_gaining_ball THEN
                        CASE
                            WHEN a.start_x < a.end_x THEN 'FORWARD'
                            WHEN a.start_x > a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    ELSE 'UNKNOWN'
                END
            -- Period 2 & 4: Teams switch sides
            WHEN a.period_id IN (2, 4) THEN
                CASE
                    -- Team starts LEFT_TO_RIGHT (now playing RIGHT_TO_LEFT in period 2)
                    WHEN plm.game_direction = 'LEFT_TO_RIGHT' AND a.team_id = plm.team_losing_ball THEN
                        CASE
                            WHEN a.start_x > a.end_x THEN 'FORWARD'
                            WHEN a.start_x < a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- Team starts RIGHT_TO_LEFT (now playing LEFT_TO_RIGHT in period 2)
                    WHEN plm.game_direction = 'RIGHT_TO_LEFT' AND a.team_id = plm.team_losing_ball THEN
                        CASE
                            WHEN a.start_x < a.end_x THEN 'FORWARD'
                            WHEN a.start_x > a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- For the team that gained possession (opposite direction)
                    WHEN plm.game_direction = 'LEFT_TO_RIGHT' AND a.team_id = plm.team_gaining_ball THEN
                        CASE
                            WHEN a.start_x < a.end_x THEN 'FORWARD'
                            WHEN a.start_x > a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    -- Team starts RIGHT_TO_LEFT
                    WHEN plm.game_direction = 'RIGHT_TO_LEFT' AND a.team_id = plm.team_gaining_ball THEN
                        CASE
                            WHEN a.start_x > a.end_x THEN 'FORWARD'
                            WHEN a.start_x < a.end_x THEN 'BACKWARD'
                            ELSE 'NEUTRAL'
                        END
                    ELSE 'UNKNOWN'
                END
            ELSE 'UNKNOWN'
        END AS movement_direction,
        -- Determine X-axis sector (divide 105m into 3 sectors)
        CASE
            WHEN a.start_x < 35 THEN 'DEFENSIVE_THIRD'
            WHEN a.start_x >= 35 AND a.start_x < 70 THEN 'MIDDLE_THIRD'
            WHEN a.start_x >= 70 THEN 'ATTACKING_THIRD'
        END AS x_sector,
        -- Determine Y-axis sector (divide 68m into 3 sectors)
        CASE
            WHEN a.start_y < 22.67 THEN 'LEFT_WING'
            WHEN a.start_y >= 22.67 AND a.start_y < 45.33 THEN 'CENTER'
            WHEN a.start_y >= 45.33 THEN 'RIGHT_WING'
        END AS y_sector
    FROM
        possession_loss_moments plm
    JOIN
        spadl_actions a ON plm.game_id = a.game_id
            AND a.period_id = plm.period_id
            AND a.seconds > plm.loss_time
            AND a.seconds <= plm.loss_time + 10
    JOIN
        players p ON a.player_id = p.player_id
    JOIN
        teams t ON a.team_id = t.team_id
),
-- Aggregated defensive success metrics per possession loss
defensive_success_summary AS (
    SELECT
        loss_event_id,
        game_id,
        period_id,
        loss_time,
        team_losing_ball,
        BOOL_OR(successful_defensive_action) AS defensive_success,
        -- Calculate recovery time if successful
        MIN(CASE WHEN successful_defensive_action THEN seconds_after_loss ELSE NULL END) AS time_to_defensive_action
    FROM
        actions_after_loss
    WHERE
        team_id = team_losing_ball  -- Only consider defensive team's actions
    GROUP BY
        loss_event_id,
        game_id,
        period_id,
        loss_time,
        team_losing_ball
)
-- Final output with both individual actions and aggregated success metrics
SELECT
    a.loss_event_id,
    a.game_id,
    a.period_id,
    a.loss_time,
    a.team_losing_ball,
    a.team_gaining_ball,
    a.loss_x,
    a.loss_y,
    a.action_id,
    a.seconds_after_loss,
    a.team_id,
    a.team_name,
    a.player_id,
    a.player_name,
    a.action_type,
    a.result,
    a.start_x,
    a.start_y,
    a.end_x,
    a.end_y,
    -- Movement and pitch position
    a.movement_direction,
    a.x_sector,
    a.y_sector,
    -- Game direction information
    a.game_direction,
    -- Defensive action indicator
    a.successful_defensive_action,
    -- Aggregated success indicators for this possession loss event
    s.defensive_success,
    s.time_to_defensive_action
FROM 
    actions_after_loss a
JOIN
    defensive_success_summary s ON a.loss_event_id = s.loss_event_id
ORDER BY
    a.game_id,
    a.period_id,
    a.loss_time,
    a.seconds_after_loss;
"""

### 2. The query is called for every team in the league and stored in csv.

In [ ]:
# Loop through all teams and execute the query for each
for _, team_row in teams_df.iterrows():
    team_id = team_row['team_id']
    team_name = team_row['team_name']
    
    print(f"Processing team: {team_name}")
    
    # Replace the placeholder with the actual team ID
    current_query = query_template.replace('{team_id}', team_id)
    
    # Execute the query
    team_data = db.execute_query(current_query)
    
    # Store the result
    all_transition_data[team_id] = team_data
    
    print(f"Found {len(team_data)} actions for {team_name}")

# Combine all data into a single DataFrame if needed
all_teams_df = pd.concat(all_transition_data.values(), ignore_index=True)


# Save the complete DataFrame to CSV
all_teams_df.to_csv('all_teams_transition_data.csv', index=False)



### 3. The CSV file is made pretty and displayed

In [ ]:
# Display the DataFrame with nice formatting
def display_pretty_table(df, num_rows=5):
    """
    Display a DataFrame as a nicely formatted table with borders.
    
    Args:
        df: pandas DataFrame to display
        num_rows: number of rows to show (like head())
    """
    print(tabulate(
        df.head(num_rows), 
        headers=df.columns, 
        tablefmt='grid',  # Use 'grid' for borders
        showindex=False   # Don't show row indices
    ))

# Display a sample of the data
print("\nSample of transition data:")
display_pretty_table(all_teams_df)

# You can also display summaries or specific slices
print("\nSummary by team:")
team_summary = all_teams_df.groupby('team_name')['action_type'].count().reset_index()
team_summary.columns = ['Team', 'Number of Actions']
display_pretty_table(team_summary)

Database connection established
Processing team: Club Brugge
Found 6650 actions for Club Brugge
Processing team: Mechelen
Found 5530 actions for Mechelen
Processing team: Sporting Charleroi
Found 6858 actions for Sporting Charleroi
Processing team: Anderlecht
Found 5911 actions for Anderlecht
Processing team: Dender
Found 6961 actions for Dender
Processing team: Kortrijk
Found 6453 actions for Kortrijk
Processing team: Beerschot
Found 5580 actions for Beerschot
Processing team: Genk
Found 6167 actions for Genk
Processing team: Westerlo
Found 7004 actions for Westerlo
Processing team: Antwerp
Found 6574 actions for Antwerp
Processing team: Gent
Found 6082 actions for Gent
Processing team: Sint-Truiden
Found 6229 actions for Sint-Truiden
Processing team: Union Saint-Gilloise
Found 7809 actions for Union Saint-Gilloise
Processing team: OH Leuven
Found 5891 actions for OH Leuven
Processing team: Cercle Brugge
Found 9191 actions for Cercle Brugge
Processing team: Standard Liège
Found 6785 a